In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("mtsamples_v1.csv")

In [5]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [6]:
df = df.drop('Unnamed: 0' , axis=1)

In [7]:
df.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [8]:
# Replacing empty strings with NaN so that it gets filled up in the function below.
df = df.replace(r'^\s*$', np.nan, regex=True)

In [9]:
# filtering out the documents that do not have keywords
filler = df[df['keywords'].isna()]

In [10]:
# close to 1000 docs dont have keywords, so using keyBERT to create keywords.
from keybert import KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [11]:
# Testing keyBERT, getting satisfactory results, so proceeding to use it for all the rows.
keywords = model.extract_keywords(filler.iloc[0]['transcription'], top_n=10)

e:\Anaconda\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [12]:
# Find the keywords.
def find_keywords(row):
    return ", ".join([word[0] for word in model.extract_keywords(docs=row['transcription'], top_n=10)])


In [13]:
# fill all keywords for each document that doesnt have keywords.
filler['keywords'] = filler.apply(find_keywords, axis = 1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3140\1616112708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filler['keywords'] = filler.apply(find_keywords, axis = 1)


In [14]:
filler.head()

,description,medical_specialty,sample_name,transcription,keywords
12,Cerebral Angiogram - moyamoya disease.,Neurology,Moyamoya Disease,"CC:, Confusion and slurred speech.,HX , (prima...","migraine, flu, vomiting, lymphadenopathy, canc..."
24,Blood in urine - Transitional cell cancer of ...,Urology,Urology Consut - 1,"CHIEF COMPLAINT:,","complaint, chief"
31,This is a 66-year-old male with signs and sym...,Urology,Urinary Retention,"CHIEF COMPLAINT:, Urinary retention.,HISTORY ...","urologist, urology, urinary, cancer, hospitali..."
32,Right distal ureteral calculus. The patient ...,Urology,Ureteral Calculus - Consult,"CHIEF COMPLAINT: , Right distal ureteral calcu...","ureteroscopy, spermatocelectomy, ureteral, rad..."
39,The patient has a possibly torsion detorsion ...,Urology,Testicular Pain,"CHIEF COMPLAINT: , Testicular pain.,HISTORY OF...","surgery, ultrasound, hospitalizations, rhinorr..."


In [15]:
# need to get the index as a column to merge on. Will remove the index later.
df2 = df.copy()
df2 = df2.reset_index()
filler2 = filler.reset_index()

In [16]:
# merged DF, this should have all the keywords present.
final_df = pd.merge(df2, filler2[['index', 'keywords']] , on='index', how='outer')

In [17]:
# Check :)
final_df.iloc[303]

index                                                              303
description           Total abdominal hysterectomy (TAH) and left s...
medical_specialty                                              Surgery
sample_name                               TAH & Salpingo-oophorectomy 
transcription        PREOPERATIVE DIAGNOSES:,1.  Hypermenorrhea.,2....
keywords_x                                                         NaN
keywords_y           surgery, hysterectomy, hemostasis, hemorrhagic...
Name: 303, dtype: object

In [18]:
# Need to merge the keywords columns.
final_df['keywords'] = final_df['keywords_x'].combine_first(final_df['keywords_y'])

In [19]:
# drop the duplicate columns and remove the index col as its no longer needed.
final_df = final_df.drop(['keywords_x', 'keywords_y', 'index'], axis=1)

In [20]:
final_df.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
final_df.to_csv('filled_mtsamples.csv')

In [ ]:
# https://pypi.org/project/names/ - used this name generator library.
import names

In [25]:
def name_generator(row):
    return "NAME : " + names.get_full_name() + "  " + row['transcription']

In [27]:
final_df['transcription'] = final_df.apply(name_generator, axis=1)

In [28]:
final_df.to_csv('synthetic_names_samples.csv')